<a href="https://colab.research.google.com/github/SARA3SAEED/aws-ML/blob/main/01_Basic_Prompt_Structure(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [40 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-1xkpohoi/wikipedia.egg-info
      writing /tmp/pip-pip-egg-info-1xkpohoi/wikipedia.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-1xkpohoi/wikipedia.egg-info/dependency_links.txt
      writing requirements to /tmp/pip-pip-egg-info-1xkpohoi/wikipedia.egg-info/requires.txt
      writing top-level names to /tmp/pip-pip-egg-info-1xkpohoi/wikipedia.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-1xkpohoi/wikipedia.egg-info/SOURCES.txt'
      reading manifest file '/tmp/pip-pip-egg-info-1xkpohoi/wikipedia.egg-info/SOURCES.txt'
      reading manifest template 'MANIFEST.in'
      adding license file 'LICENSE'
      Traceback (most recent call last):
        File "<string>", line 2

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    # Define the inference configuration
    inference_config = {
        "temperature": 0.0,  # Set the temperature for generating diverse responses
        "maxTokens": 200,  # Set the maximum number of tokens to generate
        "topP": 1,  # Set the top_p value for nucleus sampling
    }
    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,  # Specify the model ID to use
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # Provide the user's prompt
        "inferenceConfig": inference_config,  # Pass the inference configuration
    }
    # Check if system_text is provided
    if system_prompt:
        # If system_text is provided, add the system parameter to the converse_params dictionary
        converse_api_params["system"] = [{"text": system_prompt}]

    # Send a request to the Bedrock client to generate a response
    try:
        response = bedrock_client.converse(**converse_api_params)

        # Extract the generated text content from the response
        text_content = response['output']['message']['content'][0]['text']

        # Return the generated text content
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

Amazon Bedrock offers three APIs that can be used with Anthropic Claude models, the legacy [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) , the [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html) and the current [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). For this tutorial, we will be exclusively using the Converse API.

At minimum, a call to Claude using the Converse API requires the following parameters:
- `modelId`: the [API model name](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns) of the model that you intend to call

- `messages`: an array of input messages. Claude 3 models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). **The first message must always use the `user` role.**
  
  You store the content for the message in the `content` field of a [(ContentBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ContentBlock.html). Specify text in the `text` field, or if supported by the model, you can also pass the raw bytes for an image in the `image` field of an [(ImageBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ImageBlock.html). The other fields in ContentBlock are for [tool use](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html).

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

For a complete list of all API parameters, visit our [API documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html).

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The color of the ocean can vary depending on a few factors:

- Depth - The ocean appears blue in deeper waters due to the way sunlight interacts with the water molecules. Shallower waters may appear more green, turquoise, or even brown depending on the seafloor.

- Suspended particles - Things like plankton, sediment, or algae in the water can affect the ocean's color, making it appear more green, brown, or even reddish.

- Weather conditions - The sky's color can reflect on the surface of the ocean, making it appear more gray, white, or even black on overcast days.

- Geographic location - Different regions of the ocean can have slightly different hues based on factors like latitude, currents, and local marine life.

In general though, the predominant color of the open ocean is a deep, rich blue. This is due to the way sunlight interacts with the


In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Celine Dion was born on March 30, 1968.


Now let's take a look at some prompts that do not include the correct Converse API formatting. For these malformatted prompts, the Converse API returns an error.

First, we have an example of a Converse API call that lacks `role` and `content` fields in the `messages` array.

> ⚠️ **Warning:** Due to the incorrect formatting of the messages parameter in the prompt, the following cell will return an error. This is expected behavior.

In [ ]:
# Get Claude's response
inference_config = {
    "temperature": 0.0, # Set the temperature for generating diverse responses
    "maxTokens": 200 # Set the maximum number of tokens to generate
}

converse_api_params = {
    "modelId": modelId,
    "messages": [{"role":"user","content":[{"text":"Hi Claude, how are you?"}]}], # Provide the user's prompt
    "inferenceConfig": inference_config # Pass the inference configuration
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


Here's a prompt that fails to alternate between the `user` and `assistant` roles.

> ⚠️ **Warning:** Due to the lack of alternation between `user` and `assistant` roles, Claude will return an error message. This is expected behavior.

In [ ]:
inference_config = {
    "temperature": 0.5,
    "maxTokens": 200
}
# Create the converse method parameters
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
        {"role": "assistant", "content": [{"text":"Also, can you tell me some other facts about her?"}]}
    ],
    "inferenceConfig": inference_config,
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])



Celine Dion was born on March 30, 1968 in Charlemagne, Quebec, Canada.

Some other key facts about Celine Dion:

- She is a Canadian singer and businesswoman. She is one of the most successful and influential pop singers of all time.

- She began her career at a very young age, releasing her first French-language album at age 13 in 1981.

- She gained international fame in the 1990s with hit songs like "The Power of Love", "Because You Loved Me", and the theme song for the film Titanic, "My Heart Will Go On".

- She has won numerous awards over her career, including 5 Grammy Awards, 7 Billboard Music Awards, and 20 Juno Awards (Canada's top music awards).

- She has released 27 studio albums in both French and English and has


`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).

#### System Prompts

You can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn.

Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` helper function in the [Setup](#setup) section of the notebook).

Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Example

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are some critical thinking questions to further explore why the sky is blue:

- What causes the sky to appear blue in color?
- How does the composition of the Earth's atmosphere contribute to the sky's blue appearance?
- What role do the wavelengths of visible light play in making the sky appear blue?
- How do the scattering properties of different wavelengths of light affect the sky's color?
- What other factors, such as the time of day or weather conditions, can influence the sky's appearance?
- How have scientists historically explained and studied the phenomenon of the blue sky?
- What are some alternative perspectives or theories that have been proposed to explain the sky's blue color?

By exploring these types of questions, we can delve deeper into the scientific principles and mechanisms that give rise to the familiar blue hue of the sky.


Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit Anthropic's documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
Using proper `user` / `assistant` formatting, edit the `PROMPT` below to get Claude to **count to three.** The output will also indicate whether your solution is correct.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "Why is the sky blue?"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

The sky appears blue due to a phenomenon called Rayleigh scattering. Here's a brief explanation:

1. Sunlight is composed of different wavelengths of visible light, including red, orange, yellow, green, blue, and violet.

2. As sunlight passes through the Earth's atmosphere, the shorter wavelengths of light (such as blue and violet) are more easily scattered by the gas molecules in the atmosphere, particularly nitrogen and oxygen.

3. This selective scattering of the shorter wavelengths of light is known as Rayleigh scattering. The blue wavelengths are scattered more than the longer red and orange wavelengths.

4. The scattered blue light is then reflected and refracted in the atmosphere, making the sky appear blue to our eyes. This is why the sky appears more blue during the day when the sun is high in the sky.

5. At sunrise and sunset, when the sun

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_1_1_hint)

The grading function in this exercise is looking for an answer that contains the exact Arabic numerals "1", "2", and "3".
You can often get Claude to do what you want simply by asking.


### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "Respond in a playful, imaginative, and lighthearted tone."

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

The sky is absolutely massive! It's so big, it's hard to even comprehend. Just imagine - the sky stretches out in every direction as far as the eye can see. It goes on and on, with fluffy clouds drifting by and the sun or moon shining down. And that's just the part we can see from Earth! The sky actually extends far beyond our planet, all the way out into the vast expanse of the universe, with billions of stars and galaxies. It's truly mind-boggling to think about how immense and awe-inspiring the sky really is. It makes you feel so small, yet also part of something so grand and incredible. The sky is the limit, as they say - it's a boundless canvas of wonder and possibility!

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: False


❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_1_2_hint)

The grading function in this exercise is looking for answers that contain "soo" or "giggles".
There are many ways to solve this, just by asking!


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The color of the ocean can vary depending on a few factors:

- Depth - The ocean appears blue in deeper waters due to the way sunlight interacts with the water molecules. Shallower waters may appear more green, turquoise, or even brown.

- Suspended particles - Things like plankton, sediment, or algae in the water can affect the ocean's color, making it appear more green, brown, or even reddish.

- Weather conditions - The sky's color can reflect on the surface of the ocean, making it appear more gray, white, or even darker blue on overcast days.

- Geographic location - Different regions of the ocean can have slightly different hues based on factors like latitude, currents, and local marine life.

In general though, the predominant color of the open ocean is a deep, rich blue. This is due to the way sunlight interacts with the water and the lack of


In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Celine Dion was born on March 30, 1968.


In [ ]:
# Get Claude's response
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "max_tokens": 200
}

converse_api_params = {
    "modelId": modelId,
#     "messages": [{"text":"Hi Claude, how are you?"}],
     "messages": [{"role":"user","content":[{"text":"Hi Claude, how are you?"}]}],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

I'm doing well, thanks for asking! As an AI assistant, I don't have feelings in the same way humans do, but I'm functioning properly and ready to assist you. How can I help you today?


In [ ]:
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "top_p": 1,
    "max_tokens": 200
}
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
        {"role": "assistant", "content": [{"text":"Also, can you tell me some other facts about her?"}]}
    ],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])



Celine Dion was born on March 30, 1968 in Charlemagne, Quebec, Canada.

Some other key facts about Celine Dion:

- She is a Canadian singer, songwriter, and businesswoman. She is one of the most successful and influential pop singers of all time.

- She first gained international fame in the 1990s with her album "Falling Into You" and the theme song for the film Titanic, "My Heart Will Go On", which became one of the best-selling singles of all time.

- She has won numerous awards over her career, including 5 Grammy Awards, 2 Academy Awards, 7 American Music Awards, and 20 Juno Awards.

- She is the best-selling Canadian recording artist and the second best-selling female artist in history, having sold over 200 million albums worldwide.

- In addition to


In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Here are some critical thinking questions to further explore why the sky is blue:

- What is the composition of the Earth's atmosphere, and how does that affect the way light interacts with it?
- How does the scattering of different wavelengths of light contribute to the blue appearance of the sky?
- What role do gases like nitrogen and oxygen play in causing the sky to appear blue?
- How do the properties of sunlight, such as its spectrum, contribute to the blue color of the sky?
- What other factors, such as the time of day or weather conditions, can influence the appearance of the sky's color?
- How have scientists historically investigated and explained the phenomenon of the blue sky?
- What are some alternative theories or perspectives on why the sky appears blue that are worth considering?

By exploring these types of questions, we can delve deeper into the scientific principles and mechanisms that underlie the blue color of the sky.
